In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_pickle('data.pkl')
data = data[data.date_block_num > 11]
data_og = data.copy()

In [3]:
X = None
Y = None
x = None

def set_X_Y_x(df):
    global X, Y, x
    X = df[df.date_block_num < 34].drop(['item_cnt_month'], axis=1)
    Y = df[df.date_block_num < 34]['item_cnt_month']
    x = df[df.date_block_num == 34].drop(['item_cnt_month'], axis=1)

set_X_Y_x(data)

In [4]:
from sklearn.metrics import mean_squared_error

def rmse(y_te, y_p):
    return np.sqrt(mean_squared_error(y_te, y_p))

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)
lf = LinearRegression()
lf.fit(X_train, y_train)
prediction = lf.predict(X_test).clip(0,20)

og_rmse = rmse(y_test, prediction)

print(f"RMSE: {og_rmse}")

RMSE: 0.8018786024621638


In [7]:
# create correlation matrix
corr = data.corr()

# store the heavily correlated columns
corr_set = set()

# iterate over content and check where they exceed a given threshhold
for row in corr.iterrows():
    row_name = row[0]
    for entry in row[1].iteritems():
        entry_name = entry[0]
        value = abs(entry[1])
        if (value > 0.5) and (value < 1.0):
            corr_set.add(row_name)
            corr_set.add(entry_name)
            # print(f'{row_name} & {entry_name}')

# do not drop the label
corr_set.remove('item_cnt_month')

# remove it from the data column
for corr in corr_set:
    data.drop([corr], axis=1, inplace=True)

In [8]:
set_X_Y_x(data)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)
lf = LinearRegression()
lf.fit(X_train, y_train)
prediction = lf.predict(X_test).clip(0,20)

shoved_data_rmse = rmse(y_test, prediction)

print(f"RMSE: {shoved_data_rmse}")

RMSE: 0.9945539967564911


In [11]:
reduced_data = data.copy()

features_that_decrease_model = set()

for corr in corr_set:
    inspected_data = reduced_data.copy()
    inspected_data[corr] = data_og[corr]
    set_X_Y_x(inspected_data)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)
    lf = LinearRegression()
    lf.fit(X_train, y_train)
    prediction = lf.predict(X_test).clip(0,20)

    current_rmse = rmse(y_test, prediction)

    print(f"RMSE with feature {corr}: {current_rmse}")

    if current_rmse > shoved_data_rmse:
        features_that_decrease_model.add((corr, current_rmse))


RMSE with feature date_shop_avg_item_cnt_lag_1: 0.9832218707496655
RMSE with feature date_cat-medium-type_avg_item_cnt_lag_12: 0.9934251183706105
RMSE with feature date_cat-device_avg_item_cnt_lag_6: 0.9786265878115581
RMSE with feature item_cnt_month_lag_12: 0.9718415770356743
RMSE with feature date_city_avg_item_cnt_lag_3: 0.9884674636051817
RMSE with feature date_shop_cat-device_avg_item_cnt_lag_6: 0.96695410802807
RMSE with feature date_item_city_avg_item_cnt_lag_2: 0.9338404989135045
RMSE with feature delta_price_trend_with_lag_1: 0.997293434603019
RMSE with feature item_avg_price_during_month_lag_5: 0.9863512728943247
RMSE with feature item_avg_price_during_month: 0.9863072914529638
RMSE with feature date_shop_cat-device_avg_item_cnt_lag_1: 0.9463825826697974
RMSE with feature date_cat_avg_item_cnt_lag_3: 0.9746181563111309
RMSE with feature date_shop_cat-type_avg_item_cnt_lag_12: 0.9840438567518028
RMSE with feature date_cat-type_avg_item_cnt_lag_3: 0.987004735786306
RMSE with f

In [13]:
features_that_decrease_model

{('date_cat-medium-type_avg_item_cnt_lag_1', 0.9954920957070935),
 ('date_city_avg_item_cnt_lag_2', 1.000356422961999),
 ('date_shop_avg_item_cnt_lag_6', 0.9947178066114749),
 ('days_in_month', 0.9961018480254716),
 ('delta_price_trend_with_lag_1', 0.997293434603019),
 ('delta_price_trend_with_lag_4', 0.9951167564065324),
 ('delta_shop_revenue_this_month', 0.997110350972602),
 ('delta_shop_revenue_this_month_lag_2', 0.9951178173088506),
 ('item_avg_price_during_month_lag_3', 0.9956262297772916)}